# Iterative results

In [24]:
import os
import json

import pandas as pd
import numpy as np

from fault_management_uds.config import PROJ_ROOT
from fault_management_uds.config import DATA_DIR, RAW_DATA_DIR, INTERIM_DATA_DIR, PROCESSED_DATA_DIR, EXTERNAL_DATA_DIR
from fault_management_uds.config import MODELS_DIR, REPORTS_DIR, FIGURES_DIR, REFERENCE_DIR
from fault_management_uds.config import rain_gauge_color, condition_to_meta


In [ ]:
indicator_2_data_label = json.load(open(REFERENCE_DIR / 'indicator_2_data_label.json', 'r'))

In [11]:

data_types = ["train", "val", "test"]

data_type = "train"

In [ ]:
# load iterations folders
save_folder = "transformer/7_anomalous"
save_folder = MODELS_DIR / save_folder
prefix = "iteration="

relative_path = "1_split/anomalous"

filename = 'anomaly_prediction_results.pkl'


In [ ]:
def extract_outputs(outputs):
    data_label = outputs["data_label"]
    # true is where data_label is not 'Original'
    true = data_label != 'Original'
    decision_function = outputs["decision_function"]

    idx_pred_0 = outputs['0']
    label_0 = np.zeros(len(idx_pred_0), dtype=int)
    idx_pred_1 = outputs['1']
    label_1 = np.ones(len(idx_pred_1), dtype=int)

    # align these predictions with the true labels
    idx_pred = np.concatenate((idx_pred_0, idx_pred_1))
    label_pred = np.concatenate((label_0, label_1))

    # sort the predictions
    sort_indices = np.argsort(idx_pred)
    idx_pred = idx_pred[sort_indices]
    label_pred = label_pred[sort_indices]

    return idx_pred, label_pred, decision_function, true, data_label

In [ ]:

runs = os.listdir(save_folder)
runs = [run for run in runs if run.startswith(prefix)]
print(f"Total: {len(runs)}")
# take ceil of log2 of the number of runs to get the highest iteration
n_iterations = int(np.ceil(np.log2(len(runs))))
print(f"Highest iteration: {n_iterations}")

print(runs)

combined_results = {
}

for iteration in range(n_iterations):
    combined_results[str(iteration)] = {}





for run in runs:
    # get the iteration number
    iteration_identifier = run.split("=")[-1].split("_")[0]
    print(f"Iteration identifier: {iteration_identifier}")
    n_iteration = len(iteration_identifier.split("."))
    print(f"Number of iteration: {n_iteration}")

    # load the outputs file
    outputs_file = save_folder / run / relative_path / data_type / filename
    outputs = pd.read_pickle(outputs_file)

    idx_pred, label_pred, decision_function, true, data_label = extract_outputs(outputs)
    del outputs
    break
    print('')


Total: 7
Highest iteration: 3
['iteration=0_250106_0752', 'iteration=0.1_250107_1455', 'iteration=0.1.0_250108_2222', 'iteration=0.0.1_250108_2105', 'iteration=0.0_250107_1459', 'iteration=0.1.1_250108_2117', 'iteration=0.0.0_250108_2052']
Iteration identifier: 0
Number of iteration: 1
